In [6]:
import random

# RISC-V opcode definitions
OPCODES = {
    "LUI":     0b0110111,
    "AUIPC":   0b0010111,
    "JAL":     0b1101111,
    "JALR":    0b1100111,
    "BRANCH":  0b1100011,
    "LOAD":    0b0000011,
    "STORE":   0b0100011,
    "OP_IMM":  0b0010011,
    "OP":      0b0110011,
    "FENCE":   0b0001111,
    "SYSTEM":  0b1110011
}

# R-type
def gen_r(funct7, rs2, rs1, funct3, rd, opcode):
    return (funct7 << 25) | (rs2 << 20) | (rs1 << 15) | (funct3 << 12) | (rd << 7) | opcode

# I-type
def gen_i(imm, rs1, funct3, rd, opcode):
    return ((imm & 0xFFF) << 20) | (rs1 << 15) | (funct3 << 12) | (rd << 7) | opcode

# S-type
def gen_s(imm, rs2, rs1, funct3, opcode):
    imm11_5 = (imm >> 5) & 0x7F
    imm4_0 = imm & 0x1F
    return (imm11_5 << 25) | (rs2 << 20) | (rs1 << 15) | (funct3 << 12) | (imm4_0 << 7) | opcode

# B-type
def gen_b(imm, rs2, rs1, funct3, opcode):
    imm = imm >> 1
    return ((imm >> 11) & 1) << 31 | ((imm >> 1) & 0x3F) << 25 | (rs2 << 20) | (rs1 << 15) | \
           (funct3 << 12) | ((imm & 0xF) << 8) | ((imm >> 10) & 1) << 7 | opcode

# U-type
def gen_u(imm, rd, opcode):
    return (imm << 12) | (rd << 7) | opcode

# J-type
def gen_j(imm, rd, opcode):
    imm = imm >> 1
    return ((imm >> 19) & 0xFF) << 12 | ((imm >> 11) & 1) << 20 | (imm & 0x3FF) << 21 | \
           ((imm >> 10) & 1) << 31 | (rd << 7) | opcode

def generate_10_valid_instructions():
    instructions = []

    for _ in range(10):
        instr_type = random.choice([
            "ADDI", "SLTI", "SLTIU", "XORI", "ORI", "ANDI",
            "SLLI", "SRLI", "SRAI",  # I-type ALU
            "LB", "LH", "LW", "LBU", "LHU",  # Load
            "SB", "SH", "SW",  # Store
            "BEQ", "BNE", "BLT", "BGE", "BLTU", "BGEU",  # Branch
            "ADD", "SUB", "SLL", "SLT", "SLTU", "XOR", "SRL", "SRA", "OR", "AND",  # R-type
            "LUI", "AUIPC",  # U-type
            "JAL", "JALR",  # J-type
            "PAUSE", "FENCE_TSO", "EBREAK"
        ])

        rs1 = random.randint(0, 31)
        rs2 = random.randint(0, 31)
        rd = random.randint(1, 31)
        imm12 = random.randint(-2048, 2047)
        imm13 = random.choice(range(-4096, 4096, 2))
        imm20 = random.randint(0, (1 << 20) - 1)

        if instr_type == "ADDI":
            instructions.append(gen_i(imm12, rs1, 0b000, rd, OPCODES["OP_IMM"]))
        elif instr_type == "SLTI":
            instructions.append(gen_i(imm12, rs1, 0b010, rd, OPCODES["OP_IMM"]))
        elif instr_type == "SLTIU":
            instructions.append(gen_i(imm12, rs1, 0b011, rd, OPCODES["OP_IMM"]))
        elif instr_type == "XORI":
            instructions.append(gen_i(imm12, rs1, 0b100, rd, OPCODES["OP_IMM"]))
        elif instr_type == "ORI":
            instructions.append(gen_i(imm12, rs1, 0b110, rd, OPCODES["OP_IMM"]))
        elif instr_type == "ANDI":
            instructions.append(gen_i(imm12, rs1, 0b111, rd, OPCODES["OP_IMM"]))
        elif instr_type == "SLLI":
            instructions.append(gen_i(rs2 & 0x1F, rs1, 0b001, rd, OPCODES["OP_IMM"]))
        elif instr_type == "SRLI":
            instructions.append(gen_i((0b0000000 << 5) | (rs2 & 0x1F), rs1, 0b101, rd, OPCODES["OP_IMM"]))
        elif instr_type == "SRAI":
            instructions.append(gen_i((0b0100000 << 5) | (rs2 & 0x1F), rs1, 0b101, rd, OPCODES["OP_IMM"]))

        elif instr_type == "LB":
            instructions.append(gen_i(imm12, rs1, 0b000, rd, OPCODES["LOAD"]))
        elif instr_type == "LH":
            instructions.append(gen_i(imm12, rs1, 0b001, rd, OPCODES["LOAD"]))
        elif instr_type == "LW":
            instructions.append(gen_i(imm12, rs1, 0b010, rd, OPCODES["LOAD"]))
        elif instr_type == "LBU":
            instructions.append(gen_i(imm12, rs1, 0b100, rd, OPCODES["LOAD"]))
        elif instr_type == "LHU":
            instructions.append(gen_i(imm12, rs1, 0b101, rd, OPCODES["LOAD"]))

        elif instr_type == "SB":
            instructions.append(gen_s(imm12, rs2, rs1, 0b000, OPCODES["STORE"]))
        elif instr_type == "SH":
            instructions.append(gen_s(imm12, rs2, rs1, 0b001, OPCODES["STORE"]))
        elif instr_type == "SW":
            instructions.append(gen_s(imm12, rs2, rs1, 0b010, OPCODES["STORE"]))

        elif instr_type == "BEQ":
            instructions.append(gen_b(imm13, rs2, rs1, 0b000, OPCODES["BRANCH"]))
        elif instr_type == "BNE":
            instructions.append(gen_b(imm13, rs2, rs1, 0b001, OPCODES["BRANCH"]))
        elif instr_type == "BLT":
            instructions.append(gen_b(imm13, rs2, rs1, 0b100, OPCODES["BRANCH"]))
        elif instr_type == "BGE":
            instructions.append(gen_b(imm13, rs2, rs1, 0b101, OPCODES["BRANCH"]))
        elif instr_type == "BLTU":
            instructions.append(gen_b(imm13, rs2, rs1, 0b110, OPCODES["BRANCH"]))
        elif instr_type == "BGEU":
            instructions.append(gen_b(imm13, rs2, rs1, 0b111, OPCODES["BRANCH"]))

        elif instr_type == "ADD":
            instructions.append(gen_r(0b0000000, rs2, rs1, 0b000, rd, OPCODES["OP"]))
        elif instr_type == "SUB":
            instructions.append(gen_r(0b0100000, rs2, rs1, 0b000, rd, OPCODES["OP"]))
        elif instr_type == "SLL":
            instructions.append(gen_r(0b0000000, rs2, rs1, 0b001, rd, OPCODES["OP"]))
        elif instr_type == "SLT":
            instructions.append(gen_r(0b0000000, rs2, rs1, 0b010, rd, OPCODES["OP"]))
        elif instr_type == "SLTU":
            instructions.append(gen_r(0b0000000, rs2, rs1, 0b011, rd, OPCODES["OP"]))
        elif instr_type == "XOR":
            instructions.append(gen_r(0b0000000, rs2, rs1, 0b100, rd, OPCODES["OP"]))
        elif instr_type == "SRL":
            instructions.append(gen_r(0b0000000, rs2, rs1, 0b101, rd, OPCODES["OP"]))
        elif instr_type == "SRA":
            instructions.append(gen_r(0b0100000, rs2, rs1, 0b101, rd, OPCODES["OP"]))
        elif instr_type == "OR":
            instructions.append(gen_r(0b0000000, rs2, rs1, 0b110, rd, OPCODES["OP"]))
        elif instr_type == "AND":
            instructions.append(gen_r(0b0000000, rs2, rs1, 0b111, rd, OPCODES["OP"]))

        elif instr_type == "LUI":
            instructions.append(gen_u(imm20, rd, OPCODES["LUI"]))
        elif instr_type == "AUIPC":
            instructions.append(gen_u(imm20, rd, OPCODES["AUIPC"]))

        elif instr_type == "JAL":
            instructions.append(gen_j(imm20, rd, OPCODES["JAL"]))
        elif instr_type == "JALR":
            instructions.append(gen_i(imm12, rs1, 0b000, rd, OPCODES["JALR"]))

        elif instr_type == "PAUSE":
    # Use imm=0x010, which is the canonical encoding for PAUSE (pred=1, succ=0)
            instructions.append(gen_i(0x010, 0, 0b000, 0, OPCODES["FENCE"]))

        elif instr_type == "FENCE_TSO":
    # Use imm=0xFF, a valid but artificial pred/succ mask for testing
            instructions.append(gen_i(0xFF, 0, 0b000, 0, OPCODES["FENCE"]))

        elif instr_type == "EBREAK":
            instructions.append(gen_i(1, 0, 0b000, 0, OPCODES["SYSTEM"]))

    return instructions


# Write instructions to a .mem file in Verilog `32'b...` format
# Print and save as Verilog-style binary
def disassemble_and_write(filename, instructions):
    for idx, instr in enumerate(instructions):
        print(f"{idx + 1:2d}: 32'b{instr:032b}")
    with open(filename, 'w') as f:
        for instr in instructions:
            f.write(f"32'b{instr:032b}\n")

if __name__ == "__main__":
    instructions = generate_10_valid_instructions()
    write_verilog_32b("testbench_10_valid.mem", instructions)
    print("Generated testbench_10_valid.mem with 10 RV32I instructions.")


Generated testbench_10_valid.mem with 10 RV32I instructions.
